# Smooth pursuit
This is an idealized model of the smooth pursuit reflex, including two ocular muscles, a moving visual stimulus and spiking neural control.

In [ ]:
from brian2 import *
seed(79620)

Each of the two antagonistic ocular muscles is modelled as a spring of elasticity $k$ and some friction. To simplify, we consider that the eye moves laterally, rather than rotate. If $x$ is the position of the eye with 0 being the center, then the lengths of the springs are $L+x$ and $L-x$. The dynamics of the eye is then given by a second-order differential equation:
$$ m\frac{d^2x}{dt^2} = - k\left(\left(L+x\right)-x_L\right) + k\left(\left(L-x\right)-x_R\right) - f\frac{dx}{dt}$$
or:
$$ m\frac{d^2x}{dt^2} = k(x_L-x_R-2x) - f\frac{dx}{dt}$$

We see that $x_0 = \frac{1}{2}(x_L-x_R)$ is the equilibrium position of the eye. Here we have assumed that spring elasticities are identical. We can rewrite this equation with just two parameters $\alpha$ and $\beta$:
$$ \frac{d^2x}{dt^2} = \alpha(x_0 - x) - \beta\frac{dx}{dt}$$

We will assume that eye position can move between -1 and 1.

We consider that the resting length is the variable on which motoneurons act. Each spike from a motoneuron produces a waveform of contraction ("twitch"), i.e., a change in resting length. We consider that contractions add linearly, and resting lengths relax exponentially. By linearity it follows that we can simply express the action of motoneurons on the equilibrium position of the eye $x_0$, which relaxes exponentially to the center position 0.

Finally, we consider a visual object that performs a random walk according to an Ornstein–Uhlenbeck with 0 as the central location.

In [ ]:
alpha = (1/(50*ms))**2 # characteristic relaxation time is 50 ms
beta = 1/(50*ms) # friction parameter
tau_muscle = 20*ms # relaxation time of muscle contraction
tau_object = 500*ms # time constant of object movement

eqs_eye = '''
dx/dt = velocity : 1
dvelocity/dt = alpha*(x0-x)-beta*velocity : 1/second
dx0/dt = -x0/tau_muscle : 1
dx_object/dt = (noise - x_object)/tau_object:  1
dnoise/dt = -noise/tau_object + tau_object**-0.5*xi : 1
'''
eye = NeuronGroup(1, model=eqs_eye, method='euler')

We now define two motoneurons, one for each muscle:

In [ ]:
taum = 20*ms
motoneurons = NeuronGroup(2, model= 'dv/dt = -v/taum : 1', threshold = 'v>1',
                          reset = 'v=0', refractory = 5*ms, method='exact')

The motoneurons project to the eye, and each spike produces a small contraction.

In [ ]:
motosynapses = Synapses(motoneurons, eye, model = 'w : 1', on_pre = 'x0+=w')
motosynapses.connect() # connects all motoneurons to the eye
motosynapses.w = [-0.5,0.5]

We now implement the sensory neurons, which we simplify by considering spiking neurons which directly respond to light, i.e they represent both photoreceptors and retinal ganglion cells. 
TODO: Talk about Gaussian dependence, et.c

In [ ]:
N = 20
width = 2./N # width of receptive field
gain = 4.
eqs_retina = '''
I = gain*exp(-((x_object-x_eye-x_neuron)/width)**2) : 1
x_neuron : 1 (constant)
x_object : 1 (linked) # position of the object
x_eye : 1 (linked) # position of the eye
dv/dt = (I-(1+gs)*v)/taum : 1
gs : 1 # total synaptic conductance
'''
retina = NeuronGroup(N, model = eqs_retina, threshold = 'v>1', reset = 'v=0', method='exact')
retina.v = 'rand()'
retina.x_eye = linked_var(eye, 'x')
retina.x_object = linked_var(eye, 'x_object')
retina.x_neuron = '-1.0 + 2.0*i/(N-1)'

Finally we connect sensory neurons to motoneurons. Sensory neurons on each hemifield connects to the corresponding motoneuron, with a strength that scales with eccentricity:

In [ ]:
sensorimotor_synapses = Synapses(retina, motoneurons, model = 'w : 1 (constant)', on_pre = 'v+=w')
sensorimotor_synapses.connect(j = 'int(x_neuron_pre > 0)')
sensorimotor_synapses.w = '20*abs(x_neuron_pre)/N_pre'

We record the position of the eye, of the object, and spikes produced by the retina and motoneurons:

In [ ]:
M = StateMonitor(eye, ('x', 'x0', 'x_object'), record = True)
S_retina = SpikeMonitor(retina)
S_motoneurons = SpikeMonitor(motoneurons)

We now can run the simulation:

In [ ]:
run(10*second, report='text')

Finally, we plot the results

In [ ]:
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

fig = tools.make_subplots(3, 1, specs=[[{'rowspan': 2}], [None], [{}]],
                          shared_xaxes=True, print_grid=False)

trace = go.Scatter(x=S_retina.t/second,
                   y=S_retina.i,
                   marker={'symbol': 'line-ns', 'line': {'width': 1, 'color':'black'}},
                   mode='markers',
                   name='retina',
                   showlegend=False)
fig.append_trace(trace, 1, 1)
motoneuron_spikes = S_motoneurons.spike_trains()
trace = go.Scatter(x=motoneuron_spikes[0]/second,
                   y=np.ones(S_motoneurons.count[0])*N,
                   marker={'symbol': 'line-ns', 'line': {'width': 1, 'color':'#1f77b4'},
                          'color':'#1f77b4'},
                   mode='markers',
                   name='left motoneuron',
                   showlegend=False)
fig.append_trace(trace, 1, 1)
trace = go.Scatter(x=motoneuron_spikes[1]/second,
                   y=np.ones(S_motoneurons.count[1])*(N+1),
                   marker={'symbol': 'line-ns', 'line': {'width': 1, 'color':'#ff7f03'},
                           'color':'#ff7f03'},
                   mode='markers',
                   name='right motoneuron',
                   showlegend=False)
fig.append_trace(trace, 1, 1)

trace = go.Scatter(x=M.t/second,
                   y=M.x[0],
                   mode='lines',
                   line={'color': 'black'},
                   name='eye')
fig.append_trace(trace, 3, 1)
trace = go.Scatter(x=M.t/second,
                   y=M.x_object[0],
                   mode='lines',
                   line={'color': '#2ca02c'},
                   name='object')
fig.append_trace(trace, 3, 1)

fig['layout'].update(xaxis1={'showline': False,
                             'zeroline': False,
                             'title': 'time (in s)'},
                     yaxis1={'title': 'neuron index',
                             'showticklabels': False,
                             'showline': True},
                     yaxis2={'tickmode': 'array',
                             'ticktext': ['left', 'right'],
                             'tickvals': [-1, 1],
                             'range': [-1.05, 1.05],
                             'zeroline': True,
                             'showline': True}
                     )
iplot(fig)